In [181]:
%matplotlib inline
import itertools
import time
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

pd.set_option("display.max_columns", None)
ames = pd.read_stata("../data/AmesHousingGeo.dta")
ames

,longitude,latitude,pid,order,mssubclass,mszoning,lotfrontage,lotarea,street,alley,lotshape,landcontour,utilities,lotconfig,landslope,neighborhood,condition1,condition2,bldgtype,housestyle,overallqual,overallcond,yearbuilt,yearremodadd,roofstyle,roofmatl,exterior1st,exterior2nd,masvnrtype,masvnrarea,exterqual,extercond,foundation,bsmtqual,bsmtcond,bsmtexposure,bsmtfintype1,bsmtfinsf1,bsmtfintype2,bsmtfinsf2,bsmtunfsf,totalbsmtsf,heating,heatingqc,centralair,electrical,stflrsf,ndflrsf,lowqualfinsf,grlivarea,bsmtfullbath,bsmthalfbath,fullbath,halfbath,bedroomabvgr,kitchenabvgr,kitchenqual,totrmsabvgrd,functional,fireplaces,fireplacequ,garagetype,garageyrblt,garagefinish,garagecars,garagearea,garagequal,garagecond,paveddrive,wooddecksf,openporchsf,enclosedporch,ssnporch,screenporch,poolarea,poolqc,fence,miscfeature,miscval,mosold,yrsold,saletype,salecondition,saleprice,_merge
0,-93.641945,42.063297,528228325,2421,120,RL,NA,3196,Pave,NA,Reg,Lvl,AllPub,Inside,Gtl,Blmngtn,Norm,Norm,TwnhsE,1Story,7,5,2003,2004,Gable,CompShg,VinylSd,VinylSd,BrkFace,18,Gd,TA,PConc,Gd,TA,Gd,Unf,0,Unf,0,1374,1374,GasA,Ex,Y,SBrkr,1557,0,0,1557,0,0,2,0,2,1,Gd,7,Typ,1,TA,Attchd,2003,Fin,2,420,TA,TA,Y,143,20,0,0,0,0,NA,NA,NA,0,10,2006,WD,Normal,234000,matched (3)
1,-93.644920,42.011837,909452102,2227,20,RL,NA,17871,Pave,NA,IR2,Lvl,AllPub,Inside,Gtl,Crawfor,Norm,Norm,1Fam,1Story,4,5,1995,1996,Gable,CompShg,VinylSd,VinylSd,None,0,TA,TA,PConc,Gd,TA,No,Unf,0,Unf,0,1680,1680,GasA,Gd,Y,SBrkr,1680,0,0,1680,0,0,2,0,4,1,Gd,7,Typ,0,NA,Attchd,1996,Unf,2,628,TA,TA,Y,152,0,0,0,0,0,NA,NA,NA,0,6,2007,WD,Normal,170000,matched (3)
2,-93.665802,42.024124,905453080,242,20,RL,44,17485,Pave,NA,IR2,Lvl,AllPub,Inside,Gtl,Edwards,Norm,Norm,1Fam,1Story,7,5,2009,2010,Gable,CompShg,VinylSd,VinylSd,Stone,96,Gd,TA,PConc,Gd,TA,Gd,GLQ,1346,Unf,0,162,1508,GasA,Ex,Y,SBrkr,1508,0,0,1508,1,0,1,0,1,1,Gd,5,Typ,2,TA,Attchd,2009,RFn,2,572,TA,TA,Y,0,0,0,0,0,0,NA,NA,NA,0,1,2010,Con,Partial,308030,matched (3)
3,-93.687828,42.014549,907405020,1473,20,RL,70,9135,Pave,NA,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,1Story,7,5,2002,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,113,Gd,TA,PConc,Gd,TA,Av,GLQ,810,Unf,0,726,1536,GasA,Ex,Y,SBrkr,1536,0,0,1536,1,0,2,0,3,1,Gd,7,Typ,0,NA,Attchd,2002,RFn,2,532,TA,TA,Y,192,74,0,0,0,0,NA,NA,NA,0,12,2008,WD,Normal,214000,matched (3)
4,-93.689346,42.016895,907192020,2131,60,RL,68,9543,Pave,NA,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2001,Gable,CompShg,VinylSd,VinylSd,None,0,Gd,TA,PConc,Gd,TA,No,Unf,0,Unf,0,845,845,GasA,Ex,Y,SBrkr,845,845,0,1690,0,0,2,1,3,1,Gd,7,Typ,0,NA,Attchd,2001,RFn,2,517,TA,TA,Y,0,103,0,0,0,0,NA,NA,NA,0,2,2007,WD,Normal,190550,matched (3)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2914,-93.683167,42.033783,906204230,247,60,RL,87,12361,Pave,NA,IR1,Lvl,AllPub,CulDSac,Gtl,SawyerW,Norm,Norm,1Fam,2Story,6,7,1993,1993,Gable,CompShg,VinylSd,VinylSd,BrkFace,85,Gd,Gd,PConc,Gd,TA,No,GLQ,860,Unf,0,86,946,GasA,Ex,Y,SBrkr,964,838,0,1802,0,1,2,1,3,1,Gd,8,Typ,1,Gd,2Types,2000,RFn,4,1017,TA,TA,Y,450,92,0,0,0,0,NA,NA,NA,0,5,2010,WD,Normal,218000,matched (3)
2915,-93.618179,42.053326,526302040,2307,20,RL,85,10533,Pave,NA,IR1,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1956,1956,Hip,CompShg,VinylSd,VinylSd,BrkFace,244,TA,Gd,CBlock,TA,TA,No,Rec,773,Unf,0,235,1008,GasA,TA,Y,SBrkr,1024,0,0,1024,1,0,1,0,2,1,Gd,5,Typ,2,TA,Attchd,1956,Unf,1,313,TA,TA,Y,0,0,0,0,280,0,NA,NA,NA,0,8,2006,WD,Normal,157500,matched (3)
2916,-93.613853,42.019348,911175410,2880,30,C (all),69,12366,Pave,NA,Reg,Lvl,AllPub,Inside,Gtl,IDOTRR,Feedr,Norm,1Fam,1Story,3,5,1945,1950,Gable,CompShg,Wd Sdng,Wd Sdng,None,0,TA,TA,Slab,NA,NA,NA,NA,0,NA,0,0,0,GasA,TA,N,SBrkr,729,0,0

In [182]:
y = ames.saleprice
ames["qualXyear"] = ames["overallqual"] * ames["yearbuilt"]
ames["builtXstflrsf"] = ames["yearbuilt"] * ames["stflrsf"]
ames["bathXyear"] = ames["fullbath"] * ames["yearbuilt"]
ames["overallqual^2"] = ames["overallqual"]**2
ames["fullbath^2"] = ames["fullbath"]**2
ames["stflrsf^2"] = ames["stflrsf"]**2
X = ames[["overallqual","fullbath","yearbuilt","stflrsf","qualXyear","builtXstflrsf","bathXyear","overallqual^2","fullbath^2","stflrsf^2"]]
X.fillna(0)

,overallqual,fullbath,yearbuilt,stflrsf,qualXyear,builtXstflrsf,bathXyear,overallqual^2,fullbath^2,stflrsf^2
0,7,2,2003,1557,14021,-27057,4006,49,4,-583
1,4,2,1995,1680,7980,9264,3990,16,4,4352
2,7,1,2009,1508,14063,14916,2009,49,1,-19696
3,7,2,2002,1536,14014,-5120,4004,49,4,0
4,7,2,2001,845,14007,-13091,4002,49,4,-6871
...,...,...,...,...,...,...,...,...,...,...
2914,6,2,1993,964,11958,20708,3986,36,4,11792
2915,6,1,1956,1024,11736,-28672,1956,36,1,0
2916,3,1,1945,729,5835,-23887,1945,9,1,7153
2917,6,1,1954,1640,11724,-6704,1954,36,1,2624


In [183]:
def processSubset(feature_set):
    # Fit model on feature_set and calculate RSS
    model = sm.OLS(y,X[list(feature_set)])
    regr = model.fit()
    RSS = ((regr.predict(X[list(feature_set)]) - y) ** 2).sum()
    return {"model":regr, "RSS":RSS}

def process(feature_set):
    # Fit model on feature_set and calculate aic
    model = sm.OLS(y,X[list(feature_set)])
    regr = model.fit()
    return {"model":regr, "AIC":regr.aic}

In [184]:
model = sm.OLS(y,X[["overallqual","fullbath","yearbuilt","stflrsf","qualXyear","builtXstflrsf","bathXyear","overallqual^2","fullbath^2","stflrsf^2"]])
regr = model.fit()
regr.params

overallqual      -56915.257861
fullbath        -175042.220830
yearbuilt            61.079044
stflrsf              49.135568
qualXyear            13.973619
builtXstflrsf         0.088022
bathXyear            69.921789
overallqual^2      4955.360959
fullbath^2        16600.629245
stflrsf^2            -0.015931
dtype: float64

In [185]:
def getBest(k):
    
    tic = time.time()
    
    results = []
    
    for combo in itertools.combinations(X.columns, k):
        results.append(processSubset(combo))
    
    # Wrap everything up in a nice dataframe
    models = pd.DataFrame(results)
    
    # Choose the model with the smallest RSS
    best_model = models.loc[models['RSS'].argmin()]
    
    toc = time.time()
    print("Processed", models.shape[0], "models on", k, "predictors in", (toc-tic), "seconds.")
    
    # Return the best model, along with some other useful information about the model
    return best_model

def getBested(k):
    
    tic = time.time()
    
    results = []
    
    for combo in itertools.combinations(X.columns, k):
        results.append(process(combo))
    
    # Wrap everything up in a nice dataframe
    models = pd.DataFrame(results)
    
    # Choose the model with the smallest RSS
    best_model = models.loc[models['AIC'].argmin()]
    
    toc = time.time()
    print("Processed", models.shape[0], "models on", k, "predictors in", (toc-tic), "seconds.")
    
    # Return the best model, along with some other useful information about the model
    return best_model

In [186]:
models_best = pd.DataFrame(columns=["RSS", "model"])

tic = time.time()
for i in range(1,11):
    models_best.loc[i] = getBest(i)

toc = time.time()
print("Total elapsed time:", (toc-tic), "seconds.")

Processed 10 models on 1 predictors in 0.02693462371826172 seconds.
Processed 45 models on 2 predictors in 0.10949134826660156 seconds.
Processed 120 models on 3 predictors in 0.34320831298828125 seconds.
Processed 210 models on 4 predictors in 0.49741053581237793 seconds.
Processed 252 models on 5 predictors in 0.6924190521240234 seconds.
Processed 210 models on 6 predictors in 0.6543164253234863 seconds.
Processed 120 models on 7 predictors in 0.4193389415740967 seconds.
Processed 45 models on 8 predictors in 0.16251921653747559 seconds.
Processed 10 models on 9 predictors in 0.03196859359741211 seconds.
Processed 1 models on 10 predictors in 0.0061168670654296875 seconds.
Total elapsed time: 3.0041911602020264 seconds.


In [187]:
models_bested = pd.DataFrame(columns=["RSS", "model"])

tic = time.time()
for i in range(1,11):
    models_bested.loc[i] = getBested(i)

toc = time.time()
print("Total elapsed time:", (toc-tic), "seconds.")

Processed 10 models on 1 predictors in 0.0160677433013916 seconds.
Processed 45 models on 2 predictors in 0.06581974029541016 seconds.
Processed 120 models on 3 predictors in 0.20755743980407715 seconds.
Processed 210 models on 4 predictors in 0.32770228385925293 seconds.
Processed 252 models on 5 predictors in 0.5495400428771973 seconds.
Processed 210 models on 6 predictors in 0.424774169921875 seconds.
Processed 120 models on 7 predictors in 0.27910614013671875 seconds.
Processed 45 models on 8 predictors in 0.08667135238647461 seconds.
Processed 10 models on 9 predictors in 0.02236461639404297 seconds.
Processed 1 models on 10 predictors in 0.004227876663208008 seconds.
Total elapsed time: 2.045036792755127 seconds.


In [188]:
results = []

for k in range(1,17):
    for combo in itertools.combinations(X.columns, k):
        results.append(process(combo))

In [189]:
def forward(predictors):

    # Pull out predictors we still need to process
    remaining_predictors = [p for p in X.columns if p not in predictors]
    
    tic = time.time()
    
    results = []
    
    for p in remaining_predictors:
        results.append(processSubset(predictors+[p]))
    
    # Wrap everything up in a nice dataframe
    models = pd.DataFrame(results)
    
    # Choose the model with the highest RSS
    best_model = models.loc[models['RSS'].argmin()]
    
    toc = time.time()
    print("Processed ", models.shape[0], "models on", len(predictors)+1, "predictors in", (toc-tic), "seconds.")
    
    # Return the best model, along with some other useful information about the model
    return best_model

In [190]:
models_fwd = pd.DataFrame(columns=["RSS", "model"])

tic = time.time()
predictors = []

for i in range(1,len(X.columns)+1):    
    models_fwd.loc[i] = forward(predictors)
    predictors = models_fwd.loc[i]["model"].model.exog_names

toc = time.time()
print("Total elapsed time:", (toc-tic), "seconds.")

Processed  10 models on 1 predictors in 0.02486395835876465 seconds.
Processed  9 models on 2 predictors in 0.027170658111572266 seconds.
Processed  8 models on 3 predictors in 0.02255725860595703 seconds.
Processed  7 models on 4 predictors in 0.022961854934692383 seconds.
Processed  6 models on 5 predictors in 0.020298480987548828 seconds.
Processed  5 models on 6 predictors in 0.021053791046142578 seconds.
Processed  4 models on 7 predictors in 0.019704818725585938 seconds.
Processed  3 models on 8 predictors in 0.0167388916015625 seconds.
Processed  2 models on 9 predictors in 0.007486581802368164 seconds.
Processed  1 models on 10 predictors in 0.0074574947357177734 seconds.
Total elapsed time: 0.22399282455444336 seconds.


In [191]:
def backward(predictors):
    
    tic = time.time()
    
    results = []
    
    for combo in itertools.combinations(predictors, len(predictors)-1):
        results.append(processSubset(combo))
    
    # Wrap everything up in a nice dataframe
    models = pd.DataFrame(results)
    
    # Choose the model with the highest RSS
    best_model = models.loc[models['RSS'].argmin()]
    
    toc = time.time()
    print("Processed ", models.shape[0], "models on", len(predictors)-1, "predictors in", (toc-tic), "seconds.")
    
    # Return the best model, along with some other useful information about the model
    return best_model

In [192]:
models_bwd = pd.DataFrame(columns=["RSS", "model"], index = range(1,len(X.columns)))

tic = time.time()
predictors = X.columns

while(len(predictors) > 1):  
    models_bwd.loc[len(predictors)-1] = backward(predictors)
    predictors = models_bwd.loc[len(predictors)-1]["model"].model.exog_names

toc = time.time()
print("Total elapsed time:", (toc-tic), "seconds.")

Processed  10 models on 9 predictors in 0.0419003963470459 seconds.
Processed  9 models on 8 predictors in 0.03337216377258301 seconds.
Processed  8 models on 7 predictors in 0.031330108642578125 seconds.
Processed  7 models on 6 predictors in 0.027817487716674805 seconds.
Processed  6 models on 5 predictors in 0.01865243911743164 seconds.
Processed  5 models on 4 predictors in 0.015523672103881836 seconds.


Processed  4 models on 3 predictors in 0.014149665832519531 seconds.
Processed  3 models on 2 predictors in 0.01000213623046875 seconds.
Processed  2 models on 1 predictors in 0.0060198307037353516 seconds.
Total elapsed time: 0.206251859664917 seconds.


In [193]:
print("------------")
print("Best Subset:")
print("------------")
print(models_best.loc[9, "model"].params)

------------
Best Subset:
------------
overallqual      -56793.635839
fullbath        -175742.565960
yearbuilt            61.165440
stflrsf              49.111042
qualXyear            13.896037
builtXstflrsf         0.088073
bathXyear            70.247287
overallqual^2      4957.501816
fullbath^2        16623.329970
dtype: float64


In [194]:
print("-----------------")
print("Foward Selection:")
print("-----------------")
print(models_fwd.loc[9, "model"].params)

-----------------
Foward Selection:
-----------------
overallqual^2      4957.501816
stflrsf              49.111042
fullbath^2        16623.329970
fullbath        -175742.565960
bathXyear            70.247287
yearbuilt            61.165440
overallqual      -56793.635838
builtXstflrsf         0.088073
qualXyear            13.896037
dtype: float64


In [195]:
print("-------------------")
print("Backward Selection:")
print("-------------------")
print(models_bwd.loc[9, "model"].params)

-------------------
Backward Selection:
-------------------
overallqual      -56793.635839
fullbath        -175742.565960
yearbuilt            61.165440
stflrsf              49.111042
qualXyear            13.896037
builtXstflrsf         0.088073
bathXyear            70.247287
overallqual^2      4957.501816
fullbath^2        16623.329970
dtype: float64
